## Задание 1

In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score
import warnings
from sklearn.datasets import load_boston

warnings.filterwarnings("ignore")
boston = load_boston()
X = pd.DataFrame(boston["data"], columns=boston["feature_names"])
y = pd.DataFrame(boston["target"], columns=["price"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=42)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_test_preds = lr.predict(X_test)

r2 = r2_score(y_test, y_test_preds)
print("R2 score =",r2)

R2 score = 0.711226005748496


## Задание 2

In [82]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=12, 
                                 random_state=42, 
                                 n_estimators=1000
                                 )

model.fit(X_train, y_train.values[:, 0])
y_test_preds = model.predict(X_test)
r2 = r2_score(y_test, y_test_preds)
print("R2 score =",r2)

R2 score = 0.87472606157312


### Модель RandomForestRegressor работает лучше LinearRegression

## Задание 3

In [87]:
model.feature_importances_.sum()

1.0

In [114]:
fi = [(n,v)  for n,v in zip(model.feature_names_in_,model.feature_importances_)]
fi.sort(key=lambda a: a[1], reverse=True)
fi

[('LSTAT', 0.4158473181914483),
 ('RM', 0.4026817857034993),
 ('DIS', 0.06397256527230023),
 ('CRIM', 0.03167574073875602),
 ('PTRATIO', 0.01808107640502542),
 ('AGE', 0.014298644996729816),
 ('NOX', 0.014268969875348304),
 ('B', 0.0124508530140077),
 ('TAX', 0.011524934553144586),
 ('INDUS', 0.007138131415845556),
 ('RAD', 0.005281218312887839),
 ('ZN', 0.001542516626232885),
 ('CHAS', 0.0012362448947738866)]

### Наиболее важные два LSTAT RM

## Задание 4

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
df = pd.read_csv("./data/creditcard.csv", sep=',')

In [150]:
print("не сбалансирована:")
print(df.Class.value_counts(normalize=True))
df.info()
pd.options.display.max_columns = 100
df.head(10)
y = pd.DataFrame(df["Class"], columns=["Class"]) 
X = df.drop("Class", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

parameters = [{'n_estimators': [10, 15],
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}]
clf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=100),
    param_grid=parameters,
    scoring='roc_auc',
    cv=3
 )
clf.fit(X_train, y_train)

не сбалансирована:
0    0.998273
1    0.001727
Name: Class, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=100),
             param_grid=[{'max_depth': array([4, 5, 6]),
                          'max_features': array([3, 4]),
                          'n_estimators': [10, 15]}],
             scoring='roc_auc')

In [155]:
from sklearn.metrics import roc_auc_score

print(clf.best_params_)
y_pred_proba = clf.predict_proba(X_test)
y_pred_proba = y_pred_proba[:, 1]
print(roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))
print(roc_auc_score(y_test, y_pred_proba))

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}
0.9703527882554751
0.9462664156037156
